In [2]:
import re
import math
from collections import Counter

In [3]:
# Preprocess / normalize Khmer text

def normalize_khmer_spaces(text):
    """Replace invisible spaces and normalize multiple spaces"""
    text = text.replace('\u200b', ' ')  # zero-width space
    text = text.replace('\xa0', ' ')    # non-breaking space
    text = text.replace('\u200c', ' ')  # zero-width non-joiner
    text = text.replace('\u200d', ' ')  # zero-width joiner
    text = re.sub(r'\s+', ' ', text)    # replace multiple spaces with single space
    return text.strip()

def load_documents(path):
    with open(path, "r", encoding="utf-8") as f:
        docs = [line.strip() for line in f if line.strip()]
    # Normalize spaces
    docs = [normalize_khmer_spaces(doc) for doc in docs]
    return docs

docs_with_sw = load_documents("dataset_with_stopwords.txt")
docs_without_sw = load_documents("dataset_remove_stopwords.txt")



In [4]:
# Tokenize documents (split by space)
tokenized_docs_with_sw = [doc.split(" ") for doc in docs_with_sw]
tokenized_docs_without_sw = [doc.split(" ") for doc in docs_without_sw]

In [5]:
def compute_tf(doc_tokens):
    """Compute term frequency for a single document"""
    tf = {}
    word_count = Counter(doc_tokens)
    total_words = len(doc_tokens)
    for word, count in word_count.items():
        tf[word] = count / total_words
    return tf

tf_docs_with_sw = [compute_tf(doc) for doc in tokenized_docs_with_sw]
tf_docs_without_sw = [compute_tf(doc) for doc in tokenized_docs_without_sw]

In [6]:
#  Optimized IDF computation
def compute_idf_fast(tokenized_docs):
    """Compute IDF using Counter (single pass, much faster)"""
    N = len(tokenized_docs)
    df_counter = Counter()

    # Count in how many documents each word appears
    for doc in tokenized_docs:
        unique_words = set(doc)  # only count each word once per doc
        df_counter.update(unique_words)

    # Compute IDF
    idf = {word: math.log(N / (df_counter[word] + 1)) for word in df_counter}
    return idf

idf_with_sw = compute_idf_fast(tokenized_docs_with_sw)
idf_without_sw = compute_idf_fast(tokenized_docs_without_sw)

In [7]:
def compute_tfidf(tf_doc, idf):
    """Compute TF-IDF for a single document"""
    return {word: tf_value * idf.get(word, 0) for word, tf_value in tf_doc.items()}

tfidf_docs_with_sw = [compute_tfidf(tf, idf_with_sw) for tf in tf_docs_with_sw]
tfidf_docs_without_sw = [compute_tfidf(tf, idf_without_sw) for tf in tf_docs_without_sw]

In [8]:
#  Memory-efficient: Get top N words per document
def top_tfidf_words(tfidf_doc, top_n=10):
    """Return top N words and their TF-IDF scores"""
    return sorted(tfidf_doc.items(), key=lambda x: x[1], reverse=True)[:top_n]

# Example: top 10 words for first document
print("Top words with stopwords (first doc):")
for w, s in top_tfidf_words(tfidf_docs_with_sw[0]):
    print(f"{w}: {s:.4f}")

print("\nTop words without stopwords (first doc):")
for w, s in top_tfidf_words(tfidf_docs_without_sw[0]):
    print(f"{w}: {s:.4f}")

Top words with stopwords (first doc):
ឥណ្ឌា: 0.3013
របៀបវារៈ: 0.1445
ជ្រុងជ្រោយ: 0.1444
ការធ្វើដំណើរ: 0.1366
កិច្ចពិភាក្សា: 0.1366
ការអញ្ជើញ: 0.1357
សារៈសំខាន់: 0.1269
សេចក្តីថ្លែងការណ៍: 0.1197
ទាំងមូល: 0.1172
ទស្សនកិច្ច: 0.1146

Top words without stopwords (first doc):
ឥណ្ឌា: 0.5214
របៀបវារៈ: 0.2501
ជ្រុងជ្រោយ: 0.2499
ការធ្វើដំណើរ: 0.2364
កិច្ចពិភាក្សា: 0.2364
ការអញ្ជើញ: 0.2349
សារៈសំខាន់: 0.2196
សេចក្តីថ្លែងការណ៍: 0.2072
ទាំងមូល: 0.2029
ទស្សនកិច្ច: 0.1983


The TF-IDF scores increase for meaningful content words after removing stopwords.

In [11]:
from collections import defaultdict

# Compute average TF-IDF for each word across corpus
def average_tfidf(tfidf_docs):
    word_sum = defaultdict(float)
    word_count = defaultdict(int)
    
    for doc in tfidf_docs:
        for word, score in doc.items():
            word_sum[word] += score
            word_count[word] += 1
    
    avg_tfidf = {w: word_sum[w] / word_count[w] for w in word_sum}
    return dict(sorted(avg_tfidf.items(), key=lambda x: x[1], reverse=True))

avg_with_sw = average_tfidf(tfidf_docs_with_sw)
avg_without_sw = average_tfidf(tfidf_docs_without_sw)

# Display top 20 words across corpus
print("Top 20 words WITH stopwords (corpus-level):")
for w, s in list(avg_with_sw.items())[:20]:
    print(f"{w}: {s:.4f}")

print("\nTop 20 words WITHOUT stopwords (corpus-level):")
for w, s in list(avg_without_sw.items())[:20]:
    print(f"{w}: {s:.4f}")


Top 20 words WITH stopwords (corpus-level):
ឯកច្ឆន្ច: 6.6530
អសុពលភាព: 6.6530
ហ្សូអ៊ី: 5.5442
គតេ: 4.4353
ឣនុធម្មចារី: 3.6961
វេទព្វជាតក: 3.6961
កិម្បក្កជាតក: 3.6961
សុទ្ធ់: 3.6961
សីហធម្មជាតក: 3.6961
កន្ថរ៉ា: 3.6961
ហ្វហ: 3.6961
វានរជាតក: 3.6961
បច្ចុប្បន្នៈ: 3.6961
ចូច: 3.6961
ពុទ្ធារី: 3.6961
វីតា: 3.6961
ឧទាយិវគ្គ: 3.6961
អត្ថទ: 3.6961
ព្រះតន្តិ: 3.6961
ព្រះតន្តី: 3.6961

Top 20 words WITHOUT stopwords (corpus-level):
អសុពលភាព: 11.0883
ប្រូតូកាទុ: 11.0883
ត្រូវៈ: 11.0883
ឯកច្ឆន្ច: 8.3162
អ្នកសង្កេតឃើញ: 5.5442
ទៅកាន់ខ្សែភាពយន្ត: 5.5442
សាលួត: 5.5442
ឃុយ: 5.5442
អឿង: 5.5442
ដូលីន: 5.5442
បទិគះ: 5.5442
ភេញ: 5.5442
យមកៈ: 5.5442
រូបរាងៈ: 5.5442
រ៉ាដូ: 5.5442
បឹងឆ្មារ: 5.5442
ហ្សូអ៊ី: 5.5442
ភេក្នុង: 5.5442
ប៉ូកាសុី: 5.5442
ព្រះពុទ្ធញ្ញាណ: 5.5442


##### integrate Khmer stop-word dataset

In [12]:
import csv

In [13]:
STOPWORD_GROUPS = {
    "Conjunctions",
    "Pronouns",
    "Determiners & Quantifiers",
    "Prepositions / Relational Words",
    "Auxiliary Verbs / Aspect Markers",
    "Particles & Discourse Markers",
    "Question & Negation Words",
    "Function Nouns",
    "Numbers & Time Expressions",
    "Politeness & Honorifics"
}


In [14]:
def load_stopwords_from_annotated_csv(csv_path):
    stopwords = set()

    with open(csv_path, encoding="utf-8-sig") as f:
        reader = csv.DictReader(f, delimiter=",")

        print("Detected columns:", reader.fieldnames)

        for row in reader:
            term = row["term"].strip()
            group = row["linguistic_group"].strip().lower()

            # Remove everything EXCEPT content words
            if "content word" not in group:
                stopwords.add(term)

    return stopwords


KHMER_STOPWORDS = load_stopwords_from_annotated_csv("FIle_Stopwords.csv")
print("Loaded Khmer stopwords:", len(KHMER_STOPWORDS))


Detected columns: ['term', 'linguistic_group']
Loaded Khmer stopwords: 615


In [15]:
KHMER_STOPWORDS

{'កន្លែងណាមួយ',
 'កម្រិត',
 'កាន់តែ',
 'កាន់តែច្រើន',
 'ការជ្រៀតជ្រែក',
 'ការបញ្ចប់',
 'ការបើកចំហ',
 'ការផ្លាស់ប្តូរ',
 'ការរៀបចំ',
 'ការសម្រេច',
 'កាល',
 'កាលណា',
 'កាលនោះ',
 'កាលបើ',
 'កាលពី',
 'កាលពីមុន',
 'កើត',
 'កំពុង',
 'ក៏',
 'ក៏ត្រូវ',
 'ក្នុង',
 'ក្បែរ',
 'ក្បែរនេះ',
 'ក្រោម',
 'ក្រោយ',
 'ខ្លួន',
 'គាត់',
 'គួរតែ',
 'គួរសម',
 'គួរឲ្យកត់សម្គាល់',
 'គេ',
 'គេប៉ុណ្ណោះ',
 'គ្មានកន្លែង',
 'គ្មាននរណា',
 'គ្រប់',
 'គ្រប់គ្នា',
 'គ្រាន់តែ',
 'ឃើញតែ',
 'ចង់',
 'ចាប់តាំងពី',
 'ចុះ',
 'ចូល',
 'ចូលទៅ',
 'ចូលរួម',
 'ចេញ',
 'ចេញពី',
 'ចំណែក',
 'ចំណែកឯ',
 'ចំនួន',
 'ចំនួនច្រើន',
 'ចំនួនបន្ថែម',
 'ចំពោះ',
 'ចំពោះបញ្ហា',
 'ច្បាស់ណាស់',
 'ច្បាស់លាស់',
 'ច្រើន',
 'ច្រើនជាងគេ',
 'ច្រើនណាស់',
 'ឆ្លាស់គ្នា',
 'ឆ្លាស់ទៅ',
 'ឆ្លើយ',
 'ឆ្លើយឆ្លង',
 'ឆ្លើយតប',
 'ឆ្លៀតកន្លែង',
 'ឆ្លៀតការបញ្ចប់',
 'ឆ្លៀតគ្នា',
 'ឆ្លៀតឃើញ',
 'ឆ្លៀតចាប់ផ្តើម',
 'ឆ្លៀតចូលរួម',
 'ឆ្លៀតចំណេញ',
 'ឆ្លៀតចំណែក',
 'ឆ្លៀតជាមួយនឹង',
 'ឆ្លៀតដាក់',
 'ឆ្លៀតតែងតាំង',
 'ឆ្លៀតតំរង់',
 'ឆ្លៀតត្រូវ',
 'ឆ្លៀតទៅ',
 'ឆ្លៀតធ្វើ',
 'ឆ្លៀតបង្កើត

In [16]:
# Remove stopwords from tokenized docs

def remove_stopwords(tokenized_docs, stopwords):
    """Return tokenized documents with stopwords removed"""
    return [[w for w in doc if w not in stopwords] for doc in tokenized_docs]

tokenized_docs_refined = remove_stopwords(tokenized_docs_with_sw, KHMER_STOPWORDS)



In [17]:
# Recompute TF, IDF, TF-IDF for refined docs

tf_docs_refined = [compute_tf(doc) for doc in tokenized_docs_refined]
idf_refined = compute_idf_fast(tokenized_docs_refined)
tfidf_docs_refined = [compute_tfidf(tf, idf_refined) for tf in tf_docs_refined]

avg_refined = average_tfidf(tfidf_docs_refined)

In [18]:
# Display comparison

def print_top_words(title, avg_tfidf, top_n=20):
    print(f"\n{title}")
    for w, s in list(avg_tfidf.items())[:top_n]:
        print(f"{w}: {s:.4f}")

print_top_words("Top 20 words WITH original stopwords", avg_with_sw)
print_top_words("Top 20 words WITHOUT initial stopwords", avg_without_sw)
print_top_words("Top 20 words AFTER applying Khmer stopword list", avg_refined)


Top 20 words WITH original stopwords
ឯកច្ឆន្ច: 6.6530
អសុពលភាព: 6.6530
ហ្សូអ៊ី: 5.5442
គតេ: 4.4353
ឣនុធម្មចារី: 3.6961
វេទព្វជាតក: 3.6961
កិម្បក្កជាតក: 3.6961
សុទ្ធ់: 3.6961
សីហធម្មជាតក: 3.6961
កន្ថរ៉ា: 3.6961
ហ្វហ: 3.6961
វានរជាតក: 3.6961
បច្ចុប្បន្នៈ: 3.6961
ចូច: 3.6961
ពុទ្ធារី: 3.6961
វីតា: 3.6961
ឧទាយិវគ្គ: 3.6961
អត្ថទ: 3.6961
ព្រះតន្តិ: 3.6961
ព្រះតន្តី: 3.6961

Top 20 words WITHOUT initial stopwords
អសុពលភាព: 11.0883
ប្រូតូកាទុ: 11.0883
ត្រូវៈ: 11.0883
ឯកច្ឆន្ច: 8.3162
អ្នកសង្កេតឃើញ: 5.5442
ទៅកាន់ខ្សែភាពយន្ត: 5.5442
សាលួត: 5.5442
ឃុយ: 5.5442
អឿង: 5.5442
ដូលីន: 5.5442
បទិគះ: 5.5442
ភេញ: 5.5442
យមកៈ: 5.5442
រូបរាងៈ: 5.5442
រ៉ាដូ: 5.5442
បឹងឆ្មារ: 5.5442
ហ្សូអ៊ី: 5.5442
ភេក្នុង: 5.5442
ប៉ូកាសុី: 5.5442
ព្រះពុទ្ធញ្ញាណ: 5.5442

Top 20 words AFTER applying Khmer stopword list
អសុពលភាព: 11.0883
ប្រូតូកាទុ: 11.0883
ឯកច្ឆន្ច: 8.3162
សាលួត: 5.5442
ឃុយ: 5.5442
អឿង: 5.5442
ដូលីន: 5.5442
បទិគះ: 5.5442
យមកៈ: 5.5442
រូបរាងៈ: 5.5442
រ៉ាដូ: 5.5442
បឹងឆ្មារ: 5.5442
ហ្សូអ៊ី: 5.5442
ប៉ូកាសុី: 5.5

The TF-IDF computation highlights the most statistically significant words in the corpus. As expected, the top TF-IDF words are content words, proper nouns, or domain-specific terms rather than grammatical stopwords (ex: អសុពលភាព, ប្រូតូកាទុ), because true stopwords appear in almost all documents, giving them very low IDF values.

In [19]:
# Step 5: Suggest candidate stopwords (optional)
# Words still high in TF-IDF but not content words

top_candidates = [w for w, s in list(avg_refined.items())[:50]]  # top 50
print("\nCandidate stopwords to consider adding (manual review):")
print(top_candidates)



Candidate stopwords to consider adding (manual review):
['អសុពលភាព', 'ប្រូតូកាទុ', 'ឯកច្ឆន្ច', 'សាលួត', 'ឃុយ', 'អឿង', 'ដូលីន', 'បទិគះ', 'យមកៈ', 'រូបរាងៈ', 'រ៉ាដូ', 'បឹងឆ្មារ', 'ហ្សូអ៊ី', 'ប៉ូកាសុី', 'ធម្មខក្ខន្ធ', 'គីស៊ុន', 'កកសស', 'បរិប័ន្ន', 'គតេ', 'ព្រះព្រហ្មាធិរាជ', 'ព្រងិល', 'ឣនុបាទិយានោ', 'ឣនុធម្មចារី', 'វេទព្វជាតក', 'កិម្បក្កជាតក', 'សុទ្ធ់', 'សីហធម្មជាតក', 'បច្ចេកវិ', 'កន្ថរ៉ា', 'ហ្វហ', 'ហ្គ្រីឡូ', 'សទ្ទកម្ម', 'វានរជាតក', 'ប្រវត្តិការងារ', 'បច្ចុប្បន្នៈ', 'អ្នកសង្កេតឃើញ', 'ដើជ្រៃ', 'ចូច', 'ពុទ្ធារី', 'វីតា', 'ឧទាយិវគ្គ', 'អត្ថទ', 'ព្រះតន្តិ', 'ព្រះតន្តី', 'ទៅកាន់ខ្សែភាពយន្ត', 'ដារ៉', 'ម៉ាញ់', 'ឡឿក', 'បាពហុ', 'សន្ធាយ']


In [ ]:
# from collections import Counter

# def suggest_candidate_stopwords(tfidf_docs, top_n=50, min_doc_ratio=0.01):
#     """
#     Suggest candidate stopwords based on:
#     - Top N words by average TF-IDF
#     - Appear in at least `min_doc_ratio` fraction of documents
#     """
#     N_docs = len(tfidf_docs)
    
#     # Compute average TF-IDF
#     avg_tfidf = average_tfidf(tfidf_docs)
    
#     # Take top N words
#     top_words = list(avg_tfidf.keys())[:top_n]
    
#     # Count in how many documents each word appears
#     word_doc_count = Counter()
#     for doc in tfidf_docs:
#         for word in doc:
#             if word in top_words:
#                 word_doc_count[word] += 1
    
#     # Filter by document frequency ratio
#     candidates = [word for word, count in word_doc_count.items() if count / N_docs >= min_doc_ratio]
#     return candidates

# # Example
# candidate_stopwords_with_sw = suggest_candidate_stopwords(tfidf_docs_with_sw, top_n=50, min_doc_ratio=0.01)
# candidate_stopwords_without_sw = suggest_candidate_stopwords(tfidf_docs_without_sw, top_n=50, min_doc_ratio=0.01)

# print("Candidate stopwords (from WITH original stopwords):")
# print(candidate_stopwords_with_sw)

# print("\nCandidate stopwords (from WITHOUT initial stopwords):")
# print(candidate_stopwords_without_sw)


Candidate stopwords (from WITH original stopwords):
[]

Candidate stopwords (from WITHOUT initial stopwords):
[]


###### Simple IR Model Using TF-IDF and Top-K Comparison

In [21]:
import numpy as np

def cosine_similarity(vec1, vec2):
    """Compute cosine similarity between two sparse TF-IDF dicts"""
    common_words = set(vec1.keys()) & set(vec2.keys())
    num = sum(vec1[w] * vec2[w] for w in common_words)
    denom = (np.sqrt(sum(v**2 for v in vec1.values())) * 
             np.sqrt(sum(v**2 for v in vec2.values())))
    return num / denom if denom != 0 else 0

def rank_documents(query_vec, tfidf_docs, top_k=5):
    """Return top K document indices ranked by similarity"""
    scores = [(i, cosine_similarity(query_vec, doc)) for i, doc in enumerate(tfidf_docs)]
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores[:top_k]

# Example: use first document as query
query_doc = tfidf_docs_with_sw[0]

top_k_with_sw = rank_documents(query_doc, tfidf_docs_with_sw, top_k=5)
top_k_without_sw = rank_documents(query_doc, tfidf_docs_without_sw, top_k=5)

print("Top 5 similar documents WITH stopwords:")
print(top_k_with_sw)

print("\nTop 5 similar documents WITHOUT stopwords:")
print(top_k_without_sw)

# Compare overlap
indices_with_sw = set(idx for idx, _ in top_k_with_sw)
indices_without_sw = set(idx for idx, _ in top_k_without_sw)
overlap = indices_with_sw & indices_without_sw
print(f"\nOverlap in Top-5 documents: {len(overlap)} / 5")


Top 5 similar documents WITH stopwords:
[(0, 1.0000000000000002), (20044, 0.3732776475978078), (39001, 0.3665936743440593), (83277, 0.35764972611312484), (62285, 0.3469664539784011)]

Top 5 similar documents WITHOUT stopwords:
[(0, 0.9036564461309989), (20044, 0.3591125468901498), (116781, 0.3560611253905191), (62285, 0.354057607202223), (83277, 0.3520607628912101)]

Overlap in Top-5 documents: 4 / 5


We tested the impact of stopword removal on a TF-IDF based IR model. The Top-5 document retrieval using cosine similarity showed 4/5 overlap between with and without stopwords, indicating that removing stopwords slightly changed the ranking. 

This confirms that the Khmer stopword removal helps reduce noise without significantly impacting the retrieval of relevant documents.

# Apply Stop word 


In [2]:
import pandas as pd
import re
from collections import Counter
import os

# Diagnostic function to analyze text composition
def diagnose_text_issues(filepath, sample_size=100):
    """
    Diagnose what's actually in your processed text files
    """
    print(f"\n{'='*70}")
    print(f"DIAGNOSING: {os.path.basename(filepath)}")
    print(f"{'='*70}")
    
    with open(filepath, 'r', encoding='utf-8') as f:
        lines = [line.strip() for line in f if line.strip()]
    
    if not lines:
        print("⚠️  Empty file!")
        return
    
    # Sample analysis
    print(f"\n📄 Sample Documents (first 5):")
    for i, line in enumerate(lines[:5], 1):
        print(f"  {i}. {line[:200]}{'...' if len(line) > 200 else ''}")
    
    # Collect all tokens
    all_tokens = []
    for line in lines[:sample_size]:
        all_tokens.extend(line.split())
    
    if not all_tokens:
        print("⚠️  No tokens found!")
        return
    
    # Character set analysis
    khmer_pattern = re.compile(r'[\u1780-\u17FF]')  # Khmer Unicode range
    latin_pattern = re.compile(r'[a-zA-Z]')
    digit_pattern = re.compile(r'[0-9]')
    
    khmer_tokens = [t for t in all_tokens if khmer_pattern.search(t)]
    latin_tokens = [t for t in all_tokens if latin_pattern.search(t) and not khmer_pattern.search(t)]
    digit_tokens = [t for t in all_tokens if digit_pattern.search(t) and not khmer_pattern.search(t)]
    mixed_tokens = [t for t in all_tokens if khmer_pattern.search(t) and (latin_pattern.search(t) or digit_pattern.search(t))]
    
    total = len(all_tokens)
    
    print(f"\n📊 Token Composition Analysis ({total:,} tokens analyzed):")
    print(f"  • Pure Khmer tokens:    {len(khmer_tokens):6,} ({len(khmer_tokens)/total*100:5.1f}%)")
    print(f"  • Latin/Alphanumeric:   {len(latin_tokens):6,} ({len(latin_tokens)/total*100:5.1f}%)")
    print(f"  • Numeric:              {len(digit_tokens):6,} ({len(digit_tokens)/total*100:5.1f}%)")
    print(f"  • Mixed script:         {len(mixed_tokens):6,} ({len(mixed_tokens)/total*100:5.1f}%)")
    
    # Token length analysis
    token_lengths = [len(t) for t in all_tokens]
    khmer_token_lengths = [len(t) for t in khmer_tokens]
    
    print(f"\n📏 Token Length Analysis:")
    print(f"  • All tokens - Avg: {sum(token_lengths)/len(token_lengths):.1f}, Min: {min(token_lengths)}, Max: {max(token_lengths)}")
    if khmer_token_lengths:
        print(f"  • Khmer tokens - Avg: {sum(khmer_token_lengths)/len(khmer_token_lengths):.1f}, Min: {min(khmer_token_lengths)}, Max: {max(khmer_token_lengths)}")
    
    # Show suspicious tokens (too short, non-Khmer)
    print(f"\n⚠️  Suspicious Non-Khmer Tokens (sample of 20):")
    suspicious = [t for t in all_tokens if not khmer_pattern.search(t)]
    for token in Counter(suspicious).most_common(20):
        print(f"  • '{token[0]}' (appears {token[1]} times)")
    
    # Show sample Khmer tokens
    print(f"\n✓ Sample Khmer Tokens (first 20 unique):")
    khmer_unique = list(set(khmer_tokens))[:20]
    for i, token in enumerate(khmer_unique, 1):
        print(f"  {i:2d}. {token}")
    
    # Character-level breakdown of a few tokens
    print(f"\n🔬 Character-Level Analysis (first 5 non-Khmer tokens):")
    for token in suspicious[:5]:
        chars = [f"'{c}' (U+{ord(c):04X})" for c in token[:10]]
        print(f"  '{token}' → {', '.join(chars)}")
    
    return {
        'total_tokens': total,
        'khmer_tokens': len(khmer_tokens),
        'latin_tokens': len(latin_tokens),
        'digit_tokens': len(digit_tokens),
        'suspicious_tokens': len(suspicious),
        'khmer_percentage': len(khmer_tokens)/total*100 if total > 0 else 0
    }


# Run diagnostics on your files
processed_dir = r'D:\Year 5\S1\Information Retrieval\StopwordProject\khmer_stopword_project\data\processed'

files_to_check = [
    'original_segmented_sentences.txt',
    'no_all_stopwords.txt',
    'no_Auxiliary_Verbs___Aspect_Markers.txt'
]

results = {}
for filename in files_to_check:
    filepath = os.path.join(processed_dir, filename)
    if os.path.exists(filepath):
        results[filename] = diagnose_text_issues(filepath, sample_size=1000)
    else:
        print(f"\n⚠️  File not found: {filepath}")

# Summary comparison
if results:
    print(f"\n\n{'='*70}")
    print("COMPARATIVE SUMMARY")
    print(f"{'='*70}")
    print(f"\n{'File':<45} | {'Khmer %':<10} | {'Status'}")
    print(f"{'-'*45}-+-{'-'*10}-+-{'-'*20}")
    
    for filename, data in results.items():
        status = "✓ Good" if data['khmer_percentage'] > 80 else "⚠️  Problem!"
        print(f"{filename:<45} | {data['khmer_percentage']:>6.1f}%   | {status}")

print(f"\n\n{'='*70}")
print("DIAGNOSTIC COMPLETE")
print(f"{'='*70}")
print("\nNext Steps:")
print("1. Check if 'original_segmented_sentences.txt' has proper Khmer content")
print("2. If original is corrupted, review your text extraction/segmentation code")
print("3. If original is fine but filtered versions are not, review stopword removal")
print("4. Look for encoding issues in file reading/writing")


DIAGNOSING: original_segmented_sentences.txt

📄 Sample Documents (first 5):
  1. សេចក្តីថ្លែងការណ៍ ពាក់ព័ន្ធ នឹង ព័ត៌មាន នេះ   បាន ឲ្យដឹង ថា   លោក   នឹង ទៅ បំពេញ ទស្សនកិច្ច នៅ ប្រទេស ឥណ្ឌា   តាម ការអញ្ជើញ របស់ នាយក រដ្ឋមន្ត្រី ឥណ្ឌា   លោក   ដោយ ចាត់ទុក ការធ្វើដំណើរ នេះ ថា   មាន សា...
  2. ផ្តន្ទាទោស ដាក់ ពន្ធនាការ ដោយ ព្យួរ ទោស   រឺ មិន ព្យួរ ទោស ក្នុង រយ   ពេល ឆ្នាំ កន្លងទៅ ទេ   នោះ តុលាការ អាច សំរេច មិន ដាក់ទោស   ប៉ុន្តែ ត្រូវ
  3. នៅ ឆ្នាំ   ភាគ កណ្ដាល នៃ ចក្រភព បាន បែកបាក់   បង្កើត បានជា រាជាណាចក្រ តូច ឯករាជ្យ ជាច្រើន
  4. ក្នុង នោះ រួម មាន   បួរ ហ្គូញ លើ   បួរ ហ្គូញ ត្រង់ ហ្ស៊ូរ៉ាន   និង អ៊ីតាលី
  5. ចំណែកឯ តំបន់ ឡូរ៉ែ ន   ត្រូវបាន បញ្ចូល ទៅ ភាគ ខាងកើត   ក្លាយជា រាជាណាចក្រ ចំណុះ

📊 Token Composition Analysis (30,036 tokens analyzed):
  • Pure Khmer tokens:    30,036 (100.0%)
  • Latin/Alphanumeric:        0 (  0.0%)
  • Numeric:                   0 (  0.0%)
  • Mixed script:              0 (  0.0%)

📏 Token Length Analysis:
  • All tokens - Avg: 4.9, Min: 1, Max: 20
  • Khmer tok

In [3]:
import pandas as pd
import re
from collections import Counter
import os
import json

# ============================================================================
# COMPREHENSIVE KHMER TEXT DIAGNOSTIC TOOL
# ============================================================================

class KhmerTextDiagnostic:
    def __init__(self, base_dir):
        self.base_dir = base_dir
        self.khmer_pattern = re.compile(r'[\u1780-\u17FF]')  # Khmer Unicode range
        self.latin_pattern = re.compile(r'[a-zA-Z]')
        self.digit_pattern = re.compile(r'[0-9]')
        self.results = {}
        
    def analyze_file(self, filepath, label, sample_size=1000):
        """Comprehensive analysis of a single file"""
        print(f"\n{'='*80}")
        print(f"ANALYZING: {label}")
        print(f"File: {os.path.basename(filepath)}")
        print(f"{'='*80}")
        
        if not os.path.exists(filepath):
            print(f"❌ File not found: {filepath}")
            return None
        
        # Read file
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                lines = [line.strip() for line in f if line.strip()]
        except Exception as e:
            print(f"❌ Error reading file: {e}")
            return None
        
        if not lines:
            print("⚠️  Empty file - no lines to analyze!")
            return None
        
        print(f"✓ Total documents (lines): {len(lines):,}")
        
        # Show sample documents
        print(f"\n📄 SAMPLE DOCUMENTS (first 3):")
        for i, line in enumerate(lines[:3], 1):
            preview = line[:150] + '...' if len(line) > 150 else line
            print(f"  {i}. {preview}")
        
        # Tokenize sample
        all_tokens = []
        for line in lines[:sample_size]:
            all_tokens.extend(line.split())
        
        if not all_tokens:
            print("⚠️  No tokens found after splitting!")
            return None
        
        total_tokens = len(all_tokens)
        print(f"\n✓ Analyzing {total_tokens:,} tokens from first {min(sample_size, len(lines))} documents")
        
        # Categorize tokens by script
        khmer_tokens = []
        latin_tokens = []
        digit_tokens = []
        mixed_tokens = []
        other_tokens = []
        
        for token in all_tokens:
            has_khmer = bool(self.khmer_pattern.search(token))
            has_latin = bool(self.latin_pattern.search(token))
            has_digit = bool(self.digit_pattern.search(token))
            
            if has_khmer and not has_latin and not has_digit:
                khmer_tokens.append(token)
            elif has_latin and not has_khmer:
                latin_tokens.append(token)
            elif has_digit and not has_khmer and not has_latin:
                digit_tokens.append(token)
            elif has_khmer and (has_latin or has_digit):
                mixed_tokens.append(token)
            else:
                other_tokens.append(token)
        
        # Calculate percentages
        khmer_pct = (len(khmer_tokens) / total_tokens * 100) if total_tokens > 0 else 0
        latin_pct = (len(latin_tokens) / total_tokens * 100) if total_tokens > 0 else 0
        digit_pct = (len(digit_tokens) / total_tokens * 100) if total_tokens > 0 else 0
        mixed_pct = (len(mixed_tokens) / total_tokens * 100) if total_tokens > 0 else 0
        other_pct = (len(other_tokens) / total_tokens * 100) if total_tokens > 0 else 0
        
        # Display composition
        print(f"\n{'─'*80}")
        print(f"📊 TOKEN COMPOSITION ANALYSIS")
        print(f"{'─'*80}")
        print(f"{'Category':<30} | {'Count':>10} | {'Percentage':>12}")
        print(f"{'-'*30}-+-{'-'*10}-+-{'-'*12}")
        print(f"{'Pure Khmer script':<30} | {len(khmer_tokens):>10,} | {khmer_pct:>10.1f}%")
        print(f"{'Latin/Alphanumeric only':<30} | {len(latin_tokens):>10,} | {latin_pct:>10.1f}%")
        print(f"{'Numeric only':<30} | {len(digit_tokens):>10,} | {digit_pct:>10.1f}%")
        print(f"{'Mixed script':<30} | {len(mixed_tokens):>10,} | {mixed_pct:>10.1f}%")
        print(f"{'Other':<30} | {len(other_tokens):>10,} | {other_pct:>10.1f}%")
        print(f"{'-'*30}-+-{'-'*10}-+-{'-'*12}")
        print(f"{'TOTAL':<30} | {total_tokens:>10,} | {100.0:>10.1f}%")
        
        # Token length analysis
        print(f"\n{'─'*80}")
        print(f"📏 TOKEN LENGTH ANALYSIS")
        print(f"{'─'*80}")
        
        if khmer_tokens:
            khmer_lengths = [len(t) for t in khmer_tokens]
            print(f"Khmer tokens:  Avg={sum(khmer_lengths)/len(khmer_lengths):.1f}, "
                  f"Min={min(khmer_lengths)}, Max={max(khmer_lengths)}, "
                  f"Unique={len(set(khmer_tokens)):,}")
        
        if latin_tokens:
            latin_lengths = [len(t) for t in latin_tokens]
            print(f"Latin tokens:  Avg={sum(latin_lengths)/len(latin_lengths):.1f}, "
                  f"Min={min(latin_lengths)}, Max={max(latin_lengths)}, "
                  f"Unique={len(set(latin_tokens)):,}")
        
        # Show most common suspicious tokens
        print(f"\n{'─'*80}")
        print(f"⚠️  TOP 20 NON-KHMER TOKENS (Suspicious)")
        print(f"{'─'*80}")
        
        non_khmer = latin_tokens + digit_tokens + other_tokens
        if non_khmer:
            for i, (token, count) in enumerate(Counter(non_khmer).most_common(20), 1):
                print(f"{i:2d}. '{token}' → appears {count:,} times")
        else:
            print("  ✓ No non-Khmer tokens found!")
        
        # Show sample Khmer tokens
        print(f"\n{'─'*80}")
        print(f"✓ SAMPLE KHMER TOKENS (first 30 unique)")
        print(f"{'─'*80}")
        
        if khmer_tokens:
            unique_khmer = list(set(khmer_tokens))[:30]
            for i in range(0, len(unique_khmer), 5):
                row = unique_khmer[i:i+5]
                print(f"  {' | '.join(f'{t:<12}' for t in row)}")
        else:
            print("  ❌ NO KHMER TOKENS FOUND!")
        
        # Character-level analysis of suspicious tokens
        print(f"\n{'─'*80}")
        print(f"🔬 CHARACTER-LEVEL BREAKDOWN (first 5 non-Khmer tokens)")
        print(f"{'─'*80}")
        
        for token in list(set(non_khmer))[:5]:
            chars_info = []
            for char in token[:15]:  # Show first 15 chars
                chars_info.append(f"'{char}'(U+{ord(char):04X})")
            print(f"  Token: '{token}'")
            print(f"    → {', '.join(chars_info)}")
        
        # Quality assessment
        print(f"\n{'─'*80}")
        print(f"🎯 QUALITY ASSESSMENT")
        print(f"{'─'*80}")
        
        status = "✓ GOOD" if khmer_pct > 80 else "⚠️  PROBLEMATIC" if khmer_pct > 50 else "❌ CRITICAL"
        print(f"Status: {status}")
        
        if khmer_pct < 80:
            print(f"\n⚠️  ISSUES DETECTED:")
            if khmer_pct < 50:
                print(f"  • Less than 50% Khmer content - severe data quality issue")
            if latin_pct > 20:
                print(f"  • High proportion of Latin/alphanumeric tokens ({latin_pct:.1f}%)")
            if len(khmer_tokens) == 0:
                print(f"  • NO KHMER TOKENS - complete data loss!")
            if khmer_tokens and sum(len(t) for t in khmer_tokens[:100])/min(100, len(khmer_tokens)) < 3:
                print(f"  • Very short Khmer tokens - possible over-segmentation")
        
        # Store results
        result = {
            'label': label,
            'filename': os.path.basename(filepath),
            'total_documents': len(lines),
            'total_tokens': total_tokens,
            'khmer_tokens': len(khmer_tokens),
            'khmer_percentage': khmer_pct,
            'latin_tokens': len(latin_tokens),
            'latin_percentage': latin_pct,
            'digit_tokens': len(digit_tokens),
            'mixed_tokens': len(mixed_tokens),
            'unique_khmer': len(set(khmer_tokens)),
            'unique_non_khmer': len(set(non_khmer)),
            'status': status,
            'avg_khmer_length': sum(len(t) for t in khmer_tokens)/len(khmer_tokens) if khmer_tokens else 0,
            'top_non_khmer': Counter(non_khmer).most_common(10)
        }
        
        self.results[label] = result
        return result
    
    def generate_summary_report(self, output_dir):
        """Generate comprehensive summary report"""
        print(f"\n\n{'='*80}")
        print(f"GENERATING COMPREHENSIVE SUMMARY REPORT")
        print(f"{'='*80}\n")
        
        if not self.results:
            print("❌ No results to summarize!")
            return
        
        # Create summary DataFrame
        summary_data = []
        for label, result in self.results.items():
            summary_data.append({
                'Corpus': result['label'],
                'Documents': result['total_documents'],
                'Total_Tokens': result['total_tokens'],
                'Khmer_Tokens': result['khmer_tokens'],
                'Khmer_%': result['khmer_percentage'],
                'Latin_Tokens': result['latin_tokens'],
                'Latin_%': result['latin_percentage'],
                'Status': result['status']
            })
        
        df = pd.DataFrame(summary_data)
        
        # Display summary table
        print("📊 SUMMARY TABLE")
        print("─" * 120)
        print(df.to_string(index=False))
        print("─" * 120)
        
        # Save to CSV
        os.makedirs(output_dir, exist_ok=True)
        csv_file = os.path.join(output_dir, 'diagnostic_summary.csv')
        df.to_csv(csv_file, index=False, encoding='utf-8-sig')
        print(f"\n✓ Summary saved to: {csv_file}")
        
        # Generate detailed report
        report_file = os.path.join(output_dir, 'diagnostic_report.md')
        with open(report_file, 'w', encoding='utf-8') as f:
            f.write("# Khmer Corpus Diagnostic Report\n\n")
            f.write("## Executive Summary\n\n")
            
            total_files = len(self.results)
            good_files = sum(1 for r in self.results.values() if r['khmer_percentage'] > 80)
            problem_files = sum(1 for r in self.results.values() if r['khmer_percentage'] <= 80)
            critical_files = sum(1 for r in self.results.values() if r['khmer_percentage'] < 50)
            
            f.write(f"- **Total files analyzed**: {total_files}\n")
            f.write(f"- **Files with good quality (>80% Khmer)**: {good_files}\n")
            f.write(f"- **Files with issues (50-80% Khmer)**: {problem_files - critical_files}\n")
            f.write(f"- **Files with critical issues (<50% Khmer)**: {critical_files}\n\n")
            
            f.write("## Detailed Results\n\n")
            f.write(df.to_markdown(index=False))
            f.write("\n\n")
            
            f.write("## Issues and Recommendations\n\n")
            
            for label, result in self.results.items():
                if result['khmer_percentage'] < 80:
                    f.write(f"### {label}\n\n")
                    f.write(f"- **Khmer content**: {result['khmer_percentage']:.1f}%\n")
                    f.write(f"- **Top non-Khmer tokens**: {', '.join([t[0] for t in result['top_non_khmer'][:5]])}\n")
                    
                    if result['khmer_percentage'] < 50:
                        f.write(f"- **🚨 CRITICAL**: Less than 50% Khmer content\n")
                        f.write(f"- **Recommendation**: Review preprocessing pipeline completely\n")
                    elif result['avg_khmer_length'] < 3:
                        f.write(f"- **⚠️  WARNING**: Very short Khmer tokens (avg {result['avg_khmer_length']:.1f} chars)\n")
                        f.write(f"- **Recommendation**: Check tokenization algorithm\n")
                    
                    f.write("\n")
        
        print(f"✓ Detailed report saved to: {report_file}")
        
        # Generate JSON for programmatic access
        json_file = os.path.join(output_dir, 'diagnostic_results.json')
        with open(json_file, 'w', encoding='utf-8') as f:
            json.dump(self.results, f, ensure_ascii=False, indent=2)
        print(f"✓ JSON results saved to: {json_file}")
        
        # Final recommendations
        print(f"\n{'='*80}")
        print("🎯 RECOMMENDATIONS")
        print(f"{'='*80}\n")
        
        if critical_files > 0:
            print("❌ CRITICAL ISSUES FOUND:")
            print("   1. Check your preprocessing/tokenization code")
            print("   2. Verify source data is properly encoded (UTF-8)")
            print("   3. Review stopword removal - it may be too aggressive")
            print("   4. Examine raw source files for contamination\n")
        elif problem_files > 0:
            print("⚠️  SOME ISSUES FOUND:")
            print("   1. Review tokenization settings")
            print("   2. Check for metadata contamination in source")
            print("   3. Verify stopword lists are appropriate\n")
        else:
            print("✓ ALL FILES LOOK GOOD!")
            print("   Khmer content is well-preserved across all corpora.\n")

# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    # Configuration
    base_dir = r'D:\Year 5\S1\Information Retrieval\StopwordProject\khmer_stopword_project\data\processed'
    output_dir = r'D:\Year 5\S1\Information Retrieval\StopwordProject\khmer_stopword_project\data\diagnostics'
    
    # Files to analyze
    files_to_analyze = [
        ('Original', 'original_segmented_sentences.txt'),
        ('No_All_Stopwords', 'no_all_stopwords.txt'),
        ('No_Auxiliary_Verbs', 'no_Auxiliary_Verbs___Aspect_Markers.txt'),
        ('No_Conjunctions', 'no_Conjunctions.txt'),
        ('No_Determiners', 'no_Determiners_and_Quantifiers.txt'),
        ('No_Function_Nouns', 'no_Function_Nouns.txt'),
        ('No_Numbers', 'no_Numbers_and_Time_Expressions.txt'),
        ('No_Particles', 'no_Particles_and_Discourse_Markers.txt'),
        ('No_Politeness', 'no_Politeness_and_Honorifics.txt'),
        ('No_Prepositions', 'no_Prepositions___Relational_Words.txt'),
        ('No_Pronouns', 'no_Pronouns.txt'),
        ('No_Questions', 'no_Question_and_Negation_Words.txt')
    ]
    
    # Initialize diagnostic tool
    print("="*80)
    print(" KHMER TEXT DIAGNOSTIC TOOL")
    print(" Comprehensive Analysis of Processed Corpora")
    print("="*80)
    
    diagnostic = KhmerTextDiagnostic(base_dir)
    
    # Analyze each file
    for label, filename in files_to_analyze:
        filepath = os.path.join(base_dir, filename)
        diagnostic.analyze_file(filepath, label, sample_size=1000)
    
    # Generate summary report
    diagnostic.generate_summary_report(output_dir)
    
    print(f"\n{'='*80}")
    print(" DIAGNOSTIC COMPLETE")
    print(f"{'='*80}")
    print(f"\n📁 All results saved to: {output_dir}")
    print("\nFiles created:")
    print("  • diagnostic_summary.csv - Quick overview table")
    print("  • diagnostic_report.md - Detailed findings and recommendations")
    print("  • diagnostic_results.json - Complete results in JSON format")

 KHMER TEXT DIAGNOSTIC TOOL
 Comprehensive Analysis of Processed Corpora

ANALYZING: Original
File: original_segmented_sentences.txt
✓ Total documents (lines): 58,409

📄 SAMPLE DOCUMENTS (first 3):
  1. សេចក្តីថ្លែងការណ៍ ពាក់ព័ន្ធ នឹង ព័ត៌មាន នេះ   បាន ឲ្យដឹង ថា   លោក   នឹង ទៅ បំពេញ ទស្សនកិច្ច នៅ ប្រទេស ឥណ្ឌា   តាម ការអញ្ជើញ របស់ នាយក រដ្ឋមន្ត្រី ឥណ្ឌ...
  2. ផ្តន្ទាទោស ដាក់ ពន្ធនាការ ដោយ ព្យួរ ទោស   រឺ មិន ព្យួរ ទោស ក្នុង រយ   ពេល ឆ្នាំ កន្លងទៅ ទេ   នោះ តុលាការ អាច សំរេច មិន ដាក់ទោស   ប៉ុន្តែ ត្រូវ
  3. នៅ ឆ្នាំ   ភាគ កណ្ដាល នៃ ចក្រភព បាន បែកបាក់   បង្កើត បានជា រាជាណាចក្រ តូច ឯករាជ្យ ជាច្រើន

✓ Analyzing 30,036 tokens from first 1000 documents

────────────────────────────────────────────────────────────────────────────────
📊 TOKEN COMPOSITION ANALYSIS
────────────────────────────────────────────────────────────────────────────────
Category                       |      Count |   Percentage
-------------------------------+------------+-------------
Pure Khmer script              |     

In [9]:
import re
from collections import Counter
import os

# ============================================================================
# SOURCE FILE SEGMENTATION DIAGNOSTIC
# ============================================================================

def analyze_source_segmentation(filepath, num_samples=10):
    """
    Diagnose if source files have segmentation problems
    """
    print(f"\n{'='*80}")
    print(f"ANALYZING: {os.path.basename(filepath)}")
    print(f"{'='*80}\n")
    
    if not os.path.exists(filepath):
        print(f"❌ File not found: {filepath}")
        return
    
    # Read file
    with open(filepath, 'r', encoding='utf-8') as f:
        lines = [line.strip() for line in f if line.strip()]
    
    if not lines:
        print("⚠️  Empty file!")
        return
    
    print(f"Total lines: {len(lines):,}\n")
    
    # Show raw samples
    print("📄 RAW TEXT SAMPLES (first 10 lines):")
    print("─" * 80)
    for i, line in enumerate(lines[:num_samples], 1):
        preview = line[:200] + '...' if len(line) > 200 else line
        print(f"{i:2d}. {preview}")
    
    # Analyze token structure
    print(f"\n{'─'*80}")
    print("📊 TOKEN ANALYSIS")
    print("─" * 80)
    
    all_tokens = []
    for line in lines[:1000]:  # Sample first 1000 lines
        all_tokens.extend(line.split())
    
    # Categorize by length
    length_dist = Counter([len(t) for t in all_tokens])
    
    print(f"\nToken length distribution (first 1000 lines, {len(all_tokens):,} tokens):")
    for length in sorted(length_dist.keys())[:15]:  # Show first 15 lengths
        count = length_dist[length]
        pct = count / len(all_tokens) * 100
        bar = '█' * int(pct)
        print(f"  {length:2d} chars: {count:6,} ({pct:5.1f}%) {bar}")
    
    # Check for Khmer characters
    khmer_pattern = re.compile(r'[\u1780-\u17FF]')
    
    # Sample tokens by length
    print(f"\n{'─'*80}")
    print("📝 SAMPLE TOKENS BY LENGTH")
    print("─" * 80)
    
    tokens_by_length = {}
    for token in all_tokens:
        length = len(token)
        if length not in tokens_by_length:
            tokens_by_length[length] = []
        if len(tokens_by_length[length]) < 10:  # Keep max 10 samples per length
            tokens_by_length[length].append(token)
    
    for length in sorted(tokens_by_length.keys())[:8]:  # Show first 8 lengths
        samples = list(set(tokens_by_length[length]))[:10]
        print(f"\n{length} character tokens:")
        print(f"  {' | '.join(samples)}")
    
    # Statistical analysis
    print(f"\n{'─'*80}")
    print("📈 STATISTICAL SUMMARY")
    print("─" * 80)
    
    avg_length = sum(len(t) for t in all_tokens) / len(all_tokens)
    one_char = sum(1 for t in all_tokens if len(t) == 1)
    two_char = sum(1 for t in all_tokens if len(t) == 2)
    three_char = sum(1 for t in all_tokens if len(t) == 3)
    four_plus = sum(1 for t in all_tokens if len(t) >= 4)
    
    print(f"Average token length: {avg_length:.2f} characters")
    print(f"\nToken breakdown:")
    print(f"  1 char:   {one_char:7,} ({one_char/len(all_tokens)*100:5.1f}%)")
    print(f"  2 chars:  {two_char:7,} ({two_char/len(all_tokens)*100:5.1f}%)")
    print(f"  3 chars:  {three_char:7,} ({three_char/len(all_tokens)*100:5.1f}%)")
    print(f"  4+ chars: {four_plus:7,} ({four_plus/len(all_tokens)*100:5.1f}%)")
    
    # Most common tokens
    print(f"\n{'─'*80}")
    print("🔝 TOP 30 MOST COMMON TOKENS")
    print("─" * 80)
    
    token_freq = Counter(all_tokens)
    for i, (token, count) in enumerate(token_freq.most_common(30), 1):
        pct = count / len(all_tokens) * 100
        print(f"{i:2d}. '{token}' → {count:6,} times ({pct:4.1f}%) [len={len(token)}]")
    
    # Diagnosis
    print(f"\n{'─'*80}")
    print("🔍 DIAGNOSIS")
    print("─" * 80)
    
    issues = []
    
    if avg_length < 3:
        issues.append(f"❌ CRITICAL: Average token length is {avg_length:.2f} chars")
        issues.append(f"   → Tokens are TOO SHORT - likely over-segmented!")
        issues.append(f"   → Normal Khmer words should be 3-10+ characters")
    
    if one_char / len(all_tokens) > 0.2:
        issues.append(f"❌ CRITICAL: {one_char/len(all_tokens)*100:.1f}% are single characters")
        issues.append(f"   → This indicates severe segmentation problems")
    
    if two_char / len(all_tokens) > 0.3:
        issues.append(f"⚠️  WARNING: {two_char/len(all_tokens)*100:.1f}% are 2-character tokens")
        issues.append(f"   → Many tokens appear to be word fragments")
    
    if four_plus / len(all_tokens) < 0.3:
        issues.append(f"⚠️  WARNING: Only {four_plus/len(all_tokens)*100:.1f}% are 4+ characters")
        issues.append(f"   → Too few complete words")
    
    # Check if tokens look like real words
    sample_tokens = [t for t in all_tokens if len(t) >= 5][:20]
    if sample_tokens:
        print("\nSample of longer tokens (5+ chars) - do these look like real words?")
        print(f"  {', '.join(list(set(sample_tokens))[:10])}")
    
    if issues:
        print(f"\n{'='*80}")
        print("⚠️  ISSUES DETECTED:")
        print(f"{'='*80}")
        for issue in issues:
            print(issue)
        
        print(f"\n{'='*80}")
        print("💡 RECOMMENDED ACTIONS:")
        print(f"{'='*80}")
        print("1. CHECK YOUR TOKENIZATION/SEGMENTATION CODE")
        print("   → The tool that created these files is breaking words incorrectly")
        print("2. VERIFY YOUR ORIGINAL RAW TEXT")
        print("   → Go back to the source before any processing")
        print("3. USE A PROPER KHMER WORD SEGMENTER")
        print("   → Libraries: khmer-nltk, khmernltk, or PyThaiNLP (supports Khmer)")
        print("4. IF USING CUSTOM SEGMENTATION")
        print("   → Review your algorithm - it's splitting at character level, not word level")
    else:
        print("\n✅ Tokens appear to be properly segmented!")
        print("   Average length and distribution look reasonable for Khmer text.")


# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    base_dir = r'D:\Year 5\S1\Information Retrieval\StopwordProject\khmer_stopword_project\data\processed'
    
    # Files to check
    files_to_check = [
        'original_segmented_sentences.txt',
        'no_all_stopwords.txt',
        'no_Auxiliary_Verbs___Aspect_Markers.txt',
        'no_Conjunctions.txt'
    ]
    
    print("="*80)
    print(" KHMER TEXT SEGMENTATION DIAGNOSTIC TOOL")
    print(" Identifying Word Segmentation Issues")
    print("="*80)
    
    for filename in files_to_check:
        filepath = os.path.join(base_dir, filename)
        analyze_source_segmentation(filepath, num_samples=10)
        print("\n\n")
    
    print("="*80)
    print(" DIAGNOSTIC COMPLETE")
    print("="*80)
    print("\n📋 NEXT STEPS:")
    print("1. Review the diagnosis for each file above")
    print("2. If issues are found, check your preprocessing/segmentation code")
    print("3. The problem is BEFORE stopword removal - in the tokenization stage")
    print("4. You may need to re-segment your original text with a proper tool")
    print("\n💡 TIP: If 'original_segmented_sentences.txt' has issues,")
    print("   ALL downstream files will have the same problem!")

 KHMER TEXT SEGMENTATION DIAGNOSTIC TOOL
 Identifying Word Segmentation Issues

ANALYZING: original_segmented_sentences.txt

Total lines: 58,409

📄 RAW TEXT SAMPLES (first 10 lines):
────────────────────────────────────────────────────────────────────────────────
 1. សេចក្តីថ្លែងការណ៍ ពាក់ព័ន្ធ នឹង ព័ត៌មាន នេះ   បាន ឲ្យដឹង ថា   លោក   នឹង ទៅ បំពេញ ទស្សនកិច្ច នៅ ប្រទេស ឥណ្ឌា   តាម ការអញ្ជើញ របស់ នាយក រដ្ឋមន្ត្រី ឥណ្ឌា   លោក   ដោយ ចាត់ទុក ការធ្វើដំណើរ នេះ ថា   មាន សា...
 2. ផ្តន្ទាទោស ដាក់ ពន្ធនាការ ដោយ ព្យួរ ទោស   រឺ មិន ព្យួរ ទោស ក្នុង រយ   ពេល ឆ្នាំ កន្លងទៅ ទេ   នោះ តុលាការ អាច សំរេច មិន ដាក់ទោស   ប៉ុន្តែ ត្រូវ
 3. នៅ ឆ្នាំ   ភាគ កណ្ដាល នៃ ចក្រភព បាន បែកបាក់   បង្កើត បានជា រាជាណាចក្រ តូច ឯករាជ្យ ជាច្រើន
 4. ក្នុង នោះ រួម មាន   បួរ ហ្គូញ លើ   បួរ ហ្គូញ ត្រង់ ហ្ស៊ូរ៉ាន   និង អ៊ីតាលី
 5. ចំណែកឯ តំបន់ ឡូរ៉ែ ន   ត្រូវបាន បញ្ចូល ទៅ ភាគ ខាងកើត   ក្លាយជា រាជាណាចក្រ ចំណុះ
 6. ស្តេច នៃ រាជាណាចក្រ ថ្មី ទាំង នេះ   បាន ឡើង កាន់អំណាច ដោយសារ ការគាំទ្រ ពី ពួក អភិជន ក្នុង តំបន់   ប្រឆាំង នឹង អ្នក ទាមទា

In [12]:
import pandas as pd
import numpy as np
import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import json

# ============================================================================
# ENHANCED TF-IDF ANALYZER WITH QUALITY FILTERING
# ============================================================================

class KhmerTfidfAnalyzer:
    def __init__(self, processed_dir, output_dir):
        self.processed_dir = processed_dir
        self.output_dir = output_dir
        self.khmer_pattern = re.compile(r'[\u1780-\u17FF]')
        self.results = {}
        
        os.makedirs(output_dir, exist_ok=True)
    
    def diagnose_token_quality(self, documents, label):
        """Diagnose token quality before TF-IDF"""
        print(f"\n{'─'*80}")
        print(f"TOKEN QUALITY DIAGNOSTIC: {label}")
        print(f"{'─'*80}")
        
        # Sample first 100 documents
        sample_docs = documents[:100]
        all_tokens = []
        for doc in sample_docs:
            all_tokens.extend(doc.split())
        
        # Categorize tokens
        single_char = [t for t in all_tokens if len(t) == 1]
        two_char = [t for t in all_tokens if len(t) == 2]
        three_plus = [t for t in all_tokens if len(t) >= 3]
        
        print(f"Token length distribution (first 100 docs):")
        print(f"  1 character:  {len(single_char):6,} tokens ({len(single_char)/len(all_tokens)*100:5.1f}%)")
        print(f"  2 characters: {len(two_char):6,} tokens ({len(two_char)/len(all_tokens)*100:5.1f}%)")
        print(f"  3+ characters: {len(three_plus):6,} tokens ({len(three_plus)/len(all_tokens)*100:5.1f}%)")
        
        # Show most common single characters
        if single_char:
            print(f"\n  Most common single-char tokens (will be filtered):")
            for token, count in Counter(single_char).most_common(20):
                print(f"    '{token}' → {count:,} times")
        
        # Show sample of good tokens
        if three_plus:
            print(f"\n  Sample of 3+ character tokens (will be kept):")
            unique_good = list(set(three_plus))[:15]
            print(f"    {', '.join(unique_good)}")
        
        return {
            'single_char_count': len(single_char),
            'two_char_count': len(two_char),
            'three_plus_count': len(three_plus)
        }
    
    def clean_tokens(self, text, keep_khmer_only=True, min_length=2):
        """Clean text to keep only meaningful tokens"""
        tokens = text.split()
        
        if keep_khmer_only:
            # Filter criteria:
            # 1. Must contain Khmer script
            # 2. Must be at least min_length characters (default 2)
            # 3. Should not be purely non-linguistic
            cleaned_tokens = []
            
            for token in tokens:
                # Must contain Khmer
                if not self.khmer_pattern.search(token):
                    continue
                
                # Must be long enough (filter out single characters)
                if len(token) < min_length:
                    continue
                
                # Check if it's a meaningful Khmer token (not just punctuation/symbols)
                # Count actual Khmer characters (exclude combining marks, numbers, etc.)
                khmer_chars = len([c for c in token if '\u1780' <= c <= '\u17DD'])
                
                # Must have at least min_length actual Khmer base characters
                if khmer_chars < min_length:
                    continue
                
                cleaned_tokens.append(token)
            
            return ' '.join(cleaned_tokens)
        
        return text
    
    def load_and_clean_corpus(self, filepath, label):
        """Load corpus and clean it"""
        print(f"\n{'─'*80}")
        print(f"Loading: {label}")
        print(f"{'─'*80}")
        
        if not os.path.exists(filepath):
            print(f"❌ File not found: {filepath}")
            return None, None
        
        # Read documents
        with open(filepath, 'r', encoding='utf-8') as f:
            documents = [line.strip() for line in f if line.strip()]
        
        if not documents:
            print(f"⚠️  Empty file!")
            return None, None
        
        print(f"✓ Loaded {len(documents):,} documents")
        
        # Clean documents (keep only Khmer, min 2 characters)
        cleaned_docs = [self.clean_tokens(doc, keep_khmer_only=True, min_length=2) for doc in documents]
        cleaned_docs = [doc for doc in cleaned_docs if doc]  # Remove empty
        
        # Additional quality check - show sample of removed vs kept tokens
        if documents:
            sample_original = documents[0].split()[:20]
            sample_cleaned = cleaned_docs[0].split()[:20] if cleaned_docs else []
            
            removed_tokens = [t for t in sample_original if t not in ' '.join(cleaned_docs[0:5]).split()]
            
            if removed_tokens:
                print(f"  Sample removed tokens: {removed_tokens[:10]}")
            if sample_cleaned:
                print(f"  Sample kept tokens: {sample_cleaned[:10]}")
        
        # Calculate cleaning statistics
        original_tokens = sum(len(doc.split()) for doc in documents)
        cleaned_tokens = sum(len(doc.split()) for doc in cleaned_docs)
        removed_pct = ((original_tokens - cleaned_tokens) / original_tokens * 100) if original_tokens > 0 else 0
        
        print(f"  Original tokens: {original_tokens:,}")
        print(f"  After cleaning: {cleaned_tokens:,}")
        print(f"  Removed: {removed_pct:.1f}% (non-Khmer content)")
        print(f"  Documents retained: {len(cleaned_docs):,}/{len(documents):,}")
        
        if len(cleaned_docs) == 0:
            print(f"❌ No documents left after cleaning!")
            return None, None
        
        return documents, cleaned_docs
    
    def calculate_tfidf(self, documents, label, max_features=5000, min_token_length=2):
        """Calculate TF-IDF with quality checks"""
        print(f"\n{'─'*80}")
        print(f"Calculating TF-IDF: {label}")
        print(f"{'─'*80}")
        
        if not documents or len(documents) == 0:
            print("❌ No documents to analyze!")
            return None
        
        # Additional pre-filtering: remove very short tokens from documents
        filtered_docs = []
        for doc in documents:
            tokens = doc.split()
            # Keep only tokens with at least min_token_length characters
            valid_tokens = [t for t in tokens if len(t) >= min_token_length]
            if valid_tokens:
                filtered_docs.append(' '.join(valid_tokens))
        
        if not filtered_docs:
            print("❌ No valid tokens after filtering!")
            return None
        
        print(f"  Documents after token filtering: {len(filtered_docs):,}")
        
        # Create vectorizer with custom token pattern
        # This pattern ensures we only get tokens that are actual words (2+ chars)
        # CRITICAL: Use custom analyzer to prevent n-gram extraction
        def khmer_tokenizer(text):
            """Custom tokenizer that only splits on whitespace - no n-grams"""
            return text.split()
        
        vectorizer = TfidfVectorizer(
            max_features=max_features,
            min_df=2,  # Term must appear in at least 2 documents
            max_df=0.95,  # Ignore terms in >95% of documents
            tokenizer=khmer_tokenizer,  # Use our custom tokenizer
            lowercase=False,  # Don't lowercase (preserve Khmer)
            token_pattern=None  # Disable default pattern
        )
        
        try:
            # Fit and transform
            tfidf_matrix = vectorizer.fit_transform(filtered_docs)
            feature_names = vectorizer.get_feature_names_out()
            
            # Additional filtering: remove single-character terms that might have slipped through
            valid_features = []
            valid_indices = []
            for idx, term in enumerate(feature_names):
                if len(term) >= min_token_length:
                    valid_features.append(term)
                    valid_indices.append(idx)
            
            if len(valid_indices) < len(feature_names):
                print(f"  Filtered out {len(feature_names) - len(valid_indices)} single-character terms")
                # Rebuild matrix with only valid features
                tfidf_matrix = tfidf_matrix[:, valid_indices]
                feature_names = np.array(valid_features)
            
            print(f"✓ TF-IDF matrix shape: {tfidf_matrix.shape}")
            print(f"✓ Vocabulary size: {len(feature_names):,} features")
            
            # Show sample of vocabulary with length analysis
            print(f"\n  Sample vocabulary (first 20):")
            for i, term in enumerate(feature_names[:20], 1):
                print(f"    {i:2d}. '{term}' (len={len(term)})")
            
            # Analyze vocabulary by length
            vocab_lengths = Counter([len(term) for term in feature_names])
            print(f"\n  Vocabulary length distribution:")
            for length in sorted(vocab_lengths.keys())[:10]:
                count = vocab_lengths[length]
                pct = count / len(feature_names) * 100
                print(f"    {length} chars: {count:5,} terms ({pct:5.1f}%)")
            
            # Calculate statistics
            avg_tfidf_scores = np.asarray(tfidf_matrix.mean(axis=0)).flatten()
            
            # Get top terms
            top_indices = avg_tfidf_scores.argsort()[::-1][:100]
            top_terms = [(feature_names[i], avg_tfidf_scores[i]) for i in top_indices]
            
            # Verify top terms are meaningful (not single chars)
            khmer_top_terms = [(term, score) for term, score in top_terms 
                              if self.khmer_pattern.search(term) and len(term) >= 2]
            non_khmer_top_terms = [(term, score) for term, score in top_terms 
                                  if not self.khmer_pattern.search(term)]
            single_char_terms = [(term, score) for term, score in top_terms
                                if len(term) < 2]
            
            print(f"  Top 100 terms breakdown:")
            print(f"    - Khmer (2+ chars): {len(khmer_top_terms)}")
            print(f"    - Non-Khmer: {len(non_khmer_top_terms)}")
            print(f"    - Single characters: {len(single_char_terms)}")
            
            if single_char_terms:
                print(f"  ⚠️  Warning: {len(single_char_terms)} single-character terms in top 100")
                print(f"     These may indicate segmentation issues: {[t[0] for t in single_char_terms[:5]]}")
            
            if non_khmer_top_terms:
                print(f"  ⚠️  Non-Khmer terms in top 100: {[t[0] for t in non_khmer_top_terms[:5]]}")
            
            # Calculate term frequencies (from filtered docs)
            all_terms = []
            for doc in filtered_docs:
                all_terms.extend(doc.split())
            term_freq = Counter(all_terms)
            
            # Document statistics
            doc_lengths = [len(doc.split()) for doc in filtered_docs]
            
            result = {
                'label': label,
                'vectorizer': vectorizer,
                'tfidf_matrix': tfidf_matrix,
                'feature_names': feature_names,
                'avg_tfidf_scores': avg_tfidf_scores,
                'top_terms': top_terms,
                'khmer_top_terms': khmer_top_terms,
                'non_khmer_top_terms': non_khmer_top_terms,
                'single_char_terms': single_char_terms if 'single_char_terms' in locals() else [],
                'term_freq': term_freq,
                'doc_lengths': doc_lengths,
                'total_docs': len(filtered_docs),
                'total_terms': sum(doc_lengths),
                'unique_terms': len(term_freq),
                'vocab_size': len(feature_names),
                'avg_doc_length': np.mean(doc_lengths),
                'avg_tfidf': np.mean(avg_tfidf_scores),
                'std_tfidf': np.std(avg_tfidf_scores)
            }
            
            return result
            
        except Exception as e:
            print(f"❌ Error calculating TF-IDF: {e}")
            return None
    
    def save_top_terms(self, result):
        """Save top terms to file"""
        label = result['label']
        output_file = os.path.join(self.output_dir, f"top_terms_{label}.txt")
        
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(f"TOP 100 TERMS BY TF-IDF - {label}\n")
            f.write("="*80 + "\n\n")
            
            f.write(f"Documents: {result['total_docs']:,}\n")
            f.write(f"Total terms: {result['total_terms']:,}\n")
            f.write(f"Unique terms: {result['unique_terms']:,}\n")
            f.write(f"Vocabulary size: {result['vocab_size']:,}\n")
            f.write(f"Avg TF-IDF score: {result['avg_tfidf']:.6f}\n\n")
            
            # Separate Khmer and non-Khmer terms
            f.write("="*80 + "\n")
            f.write("KHMER TERMS (Pure Khmer script only)\n")
            f.write("="*80 + "\n")
            f.write(f"{'Rank':<6} | {'Term':<20} | {'Avg TF-IDF Score':<18}\n")
            f.write("-"*80 + "\n")
            
            for i, (term, score) in enumerate(result['khmer_top_terms'][:100], 1):
                f.write(f"{i:<6} | {term:<20} | {score:<18.6f}\n")
            
            if result['non_khmer_top_terms']:
                f.write("\n" + "="*80 + "\n")
                f.write("NON-KHMER TERMS (Suspicious - may indicate data quality issues)\n")
                f.write("="*80 + "\n")
                f.write(f"{'Rank':<6} | {'Term':<20} | {'Avg TF-IDF Score':<18}\n")
                f.write("-"*80 + "\n")
                
                for i, (term, score) in enumerate(result['non_khmer_top_terms'][:50], 1):
                    f.write(f"{i:<6} | {term:<20} | {score:<18.6f}\n")
        
        print(f"✓ Saved top terms to: {os.path.basename(output_file)}")
    
    def analyze_all_files(self, files_to_analyze):
        """Analyze all corpus files"""
        print("\n" + "="*80)
        print(" COMPREHENSIVE TF-IDF ANALYSIS")
        print("="*80)
        
        for label, filename in files_to_analyze:
            filepath = os.path.join(self.processed_dir, filename)
            
            # Load and clean
            original_docs, cleaned_docs = self.load_and_clean_corpus(filepath, label)
            
            if cleaned_docs is None:
                continue
            
            # Diagnose token quality
            self.diagnose_token_quality(cleaned_docs, label)
            
            # Calculate TF-IDF
            result = self.calculate_tfidf(cleaned_docs, label)
            
            if result is None:
                continue
            
            # Save results
            self.results[label] = result
            self.save_top_terms(result)
            
            # Save TF-IDF matrix (sparse format)
            matrix_file = os.path.join(self.output_dir, f"tfidf_matrix_{label}.npz")
            np.savez(matrix_file,
                    data=result['tfidf_matrix'].data,
                    indices=result['tfidf_matrix'].indices,
                    indptr=result['tfidf_matrix'].indptr,
                    shape=result['tfidf_matrix'].shape)
            
            # Save feature names
            features_file = os.path.join(self.output_dir, f"features_{label}.txt")
            with open(features_file, 'w', encoding='utf-8') as f:
                for feature in result['feature_names']:
                    f.write(feature + '\n')
    
    def generate_comparative_analysis(self):
        """Generate comparative analysis across all corpora"""
        print("\n" + "="*80)
        print(" COMPARATIVE ANALYSIS")
        print("="*80)
        
        if len(self.results) < 2:
            print("⚠️  Need at least 2 corpora for comparison")
            return
        
        # 1. Summary statistics
        print("\n📊 SUMMARY STATISTICS")
        summary_data = []
        for label, result in self.results.items():
            summary_data.append({
                'Corpus': label,
                'Documents': result['total_docs'],
                'Total_Terms': result['total_terms'],
                'Unique_Terms': result['unique_terms'],
                'Vocab_Size': result['vocab_size'],
                'Avg_Doc_Length': result['avg_doc_length'],
                'Avg_TF-IDF': result['avg_tfidf'],
                'Khmer_Top_Terms': len(result['khmer_top_terms']),
                'Non-Khmer_Top_Terms': len(result['non_khmer_top_terms'])
            })
        
        summary_df = pd.DataFrame(summary_data)
        print("\n" + summary_df.to_string(index=False))
        
        # Save summary
        summary_file = os.path.join(self.output_dir, 'tfidf_summary.csv')
        summary_df.to_csv(summary_file, index=False, encoding='utf-8-sig')
        print(f"\n✓ Summary saved to: {os.path.basename(summary_file)}")
        
        # 2. Vocabulary overlap analysis
        print("\n📊 VOCABULARY OVERLAP ANALYSIS")
        vocabularies = {label: set(result['feature_names']) 
                       for label, result in self.results.items()}
        
        labels = list(vocabularies.keys())
        overlap_matrix = np.zeros((len(labels), len(labels)))
        
        for i, label1 in enumerate(labels):
            for j, label2 in enumerate(labels):
                if i == j:
                    overlap_matrix[i, j] = 1.0
                else:
                    intersection = len(vocabularies[label1] & vocabularies[label2])
                    union = len(vocabularies[label1] | vocabularies[label2])
                    overlap_matrix[i, j] = intersection / union if union > 0 else 0
        
        overlap_df = pd.DataFrame(overlap_matrix, index=labels, columns=labels)
        overlap_file = os.path.join(self.output_dir, 'vocabulary_overlap.csv')
        overlap_df.to_csv(overlap_file, encoding='utf-8-sig')
        print(f"✓ Vocabulary overlap saved to: {os.path.basename(overlap_file)}")
        
        # 3. Top terms comparison
        print("\n📊 TOP TERMS COMPARISON")
        comparison_file = os.path.join(self.output_dir, 'top_terms_comparison.txt')
        
        with open(comparison_file, 'w', encoding='utf-8') as f:
            f.write("TOP 20 KHMER TERMS COMPARISON\n")
            f.write("="*120 + "\n\n")
            
            # Get top 20 Khmer terms from each corpus
            top_terms_dict = {}
            for label, result in self.results.items():
                top_terms_dict[label] = [term for term, _ in result['khmer_top_terms'][:20]]
            
            # Create comparison table
            f.write(f"{'Rank':<6}")
            for label in top_terms_dict.keys():
                f.write(f" | {label:<25}")
            f.write("\n" + "-"*120 + "\n")
            
            for i in range(20):
                f.write(f"{i+1:<6}")
                for label in top_terms_dict.keys():
                    term = top_terms_dict[label][i] if i < len(top_terms_dict[label]) else ""
                    f.write(f" | {term:<25}")
                f.write("\n")
        
        print(f"✓ Top terms comparison saved to: {os.path.basename(comparison_file)}")
        
        # 4. Impact analysis (compared to Original)
        if 'Original' in self.results:
            print("\n📊 IMPACT OF STOPWORD REMOVAL")
            original = self.results['Original']
            
            impact_data = []
            for label, result in self.results.items():
                if label == 'Original':
                    continue
                
                term_reduction = (1 - result['total_terms'] / original['total_terms']) * 100
                unique_reduction = (1 - result['unique_terms'] / original['unique_terms']) * 100
                vocab_reduction = (1 - result['vocab_size'] / original['vocab_size']) * 100
                
                impact_data.append({
                    'Stopword_Group': label.replace('No_', ''),
                    'Term_Reduction_%': term_reduction,
                    'Unique_Term_Reduction_%': unique_reduction,
                    'Vocab_Reduction_%': vocab_reduction,
                    'Remaining_Terms': result['total_terms'],
                    'Quality_Score': len(result['khmer_top_terms']) / 100 * 100  # % Khmer in top 100
                })
            
            impact_df = pd.DataFrame(impact_data)
            impact_df = impact_df.sort_values('Term_Reduction_%', ascending=False)
            
            print("\n" + impact_df.to_string(index=False))
            
            impact_file = os.path.join(self.output_dir, 'stopword_impact.csv')
            impact_df.to_csv(impact_file, index=False, encoding='utf-8-sig')
            print(f"\n✓ Impact analysis saved to: {os.path.basename(impact_file)}")
    
    def create_visualizations(self):
        """Create comprehensive visualizations"""
        print("\n📊 CREATING VISUALIZATIONS")
        
        plt.style.use('seaborn-v0_8-darkgrid')
        
        # Figure 1: Corpus statistics
        fig1, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig1.suptitle('Corpus Statistics Comparison', fontsize=16, fontweight='bold')
        
        labels = list(self.results.keys())
        total_terms = [self.results[l]['total_terms'] for l in labels]
        unique_terms = [self.results[l]['unique_terms'] for l in labels]
        vocab_sizes = [self.results[l]['vocab_size'] for l in labels]
        avg_lengths = [self.results[l]['avg_doc_length'] for l in labels]
        
        # Plot 1: Total terms
        axes[0, 0].bar(range(len(labels)), total_terms, color='skyblue', edgecolor='black')
        axes[0, 0].set_xticks(range(len(labels)))
        axes[0, 0].set_xticklabels(labels, rotation=45, ha='right')
        axes[0, 0].set_ylabel('Number of Terms', fontsize=12)
        axes[0, 0].set_title('Total Terms per Corpus', fontsize=14, fontweight='bold')
        axes[0, 0].grid(axis='y', alpha=0.3)
        
        # Plot 2: Unique terms
        axes[0, 1].bar(range(len(labels)), unique_terms, color='lightgreen', edgecolor='black')
        axes[0, 1].set_xticks(range(len(labels)))
        axes[0, 1].set_xticklabels(labels, rotation=45, ha='right')
        axes[0, 1].set_ylabel('Number of Unique Terms', fontsize=12)
        axes[0, 1].set_title('Unique Terms per Corpus', fontsize=14, fontweight='bold')
        axes[0, 1].grid(axis='y', alpha=0.3)
        
        # Plot 3: Vocabulary size
        axes[1, 0].bar(range(len(labels)), vocab_sizes, color='salmon', edgecolor='black')
        axes[1, 0].set_xticks(range(len(labels)))
        axes[1, 0].set_xticklabels(labels, rotation=45, ha='right')
        axes[1, 0].set_ylabel('Vocabulary Size', fontsize=12)
        axes[1, 0].set_title('TF-IDF Vocabulary Size', fontsize=14, fontweight='bold')
        axes[1, 0].grid(axis='y', alpha=0.3)
        
        # Plot 4: Average document length
        axes[1, 1].bar(range(len(labels)), avg_lengths, color='gold', edgecolor='black')
        axes[1, 1].set_xticks(range(len(labels)))
        axes[1, 1].set_xticklabels(labels, rotation=45, ha='right')
        axes[1, 1].set_ylabel('Average Terms per Document', fontsize=12)
        axes[1, 1].set_title('Average Document Length', fontsize=14, fontweight='bold')
        axes[1, 1].grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        fig1.savefig(os.path.join(self.output_dir, 'corpus_statistics.png'), 
                    dpi=300, bbox_inches='tight')
        print("✓ Saved: corpus_statistics.png")
        
        # Figure 2: TF-IDF distributions
        n_corpora = len(self.results)
        n_cols = 4
        n_rows = (n_corpora + n_cols - 1) // n_cols
        
        fig2, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5*n_rows))
        fig2.suptitle('TF-IDF Score Distributions', fontsize=16, fontweight='bold')
        
        if n_rows == 1:
            axes = axes.reshape(1, -1)
        
        for idx, (label, result) in enumerate(self.results.items()):
            row, col = divmod(idx, n_cols)
            ax = axes[row, col]
            
            scores = result['avg_tfidf_scores']
            ax.hist(scores, bins=50, alpha=0.7, color='steelblue', edgecolor='black')
            ax.axvline(np.mean(scores), color='red', linestyle='--', linewidth=2,
                      label=f'Mean: {np.mean(scores):.4f}')
            ax.set_xlabel('TF-IDF Score', fontsize=10)
            ax.set_ylabel('Frequency', fontsize=10)
            ax.set_title(label, fontsize=12, fontweight='bold')
            ax.legend()
            ax.grid(alpha=0.3)
        
        # Remove empty subplots
        for idx in range(n_corpora, n_rows * n_cols):
            row, col = divmod(idx, n_cols)
            fig2.delaxes(axes[row, col])
        
        plt.tight_layout()
        fig2.savefig(os.path.join(self.output_dir, 'tfidf_distributions.png'),
                    dpi=300, bbox_inches='tight')
        print("✓ Saved: tfidf_distributions.png")
        
        # Figure 3: Quality scores (Khmer vs Non-Khmer in top 100)
        fig3, ax = plt.subplots(figsize=(14, 8))
        
        khmer_counts = [len(self.results[l]['khmer_top_terms']) for l in labels]
        non_khmer_counts = [len(self.results[l]['non_khmer_top_terms']) for l in labels]
        
        x = np.arange(len(labels))
        width = 0.35
        
        ax.bar(x - width/2, khmer_counts, width, label='Khmer Terms', 
               color='green', alpha=0.7, edgecolor='black')
        ax.bar(x + width/2, non_khmer_counts, width, label='Non-Khmer Terms',
               color='red', alpha=0.7, edgecolor='black')
        
        ax.set_xlabel('Corpus', fontsize=12, fontweight='bold')
        ax.set_ylabel('Count in Top 100', fontsize=12, fontweight='bold')
        ax.set_title('Quality Check: Khmer vs Non-Khmer Terms in Top 100',
                    fontsize=14, fontweight='bold')
        ax.set_xticks(x)
        ax.set_xticklabels(labels, rotation=45, ha='right')
        ax.legend()
        ax.grid(axis='y', alpha=0.3)
        ax.axhline(y=80, color='orange', linestyle='--', label='Target: 80+ Khmer terms')
        
        plt.tight_layout()
        fig3.savefig(os.path.join(self.output_dir, 'quality_check.png'),
                    dpi=300, bbox_inches='tight')
        print("✓ Saved: quality_check.png")
        
        plt.close('all')
    
    def generate_final_report(self):
        """Generate comprehensive final report"""
        print("\n📄 GENERATING FINAL REPORT")
        
        report_file = os.path.join(self.output_dir, 'tfidf_analysis_report.md')
        
        with open(report_file, 'w', encoding='utf-8') as f:
            f.write("# Comprehensive TF-IDF Analysis Report\n\n")
            f.write("## Executive Summary\n\n")
            
            total_corpora = len(self.results)
            f.write(f"- **Total corpora analyzed**: {total_corpora}\n")
            
            if 'Original' in self.results:
                orig = self.results['Original']
                f.write(f"- **Original corpus**: {orig['total_docs']:,} documents, "
                       f"{orig['total_terms']:,} terms\n")
            
            # Quality assessment
            high_quality = sum(1 for r in self.results.values() 
                             if len(r['khmer_top_terms']) >= 80)
            f.write(f"- **High quality corpora** (≥80 Khmer terms in top 100): {high_quality}/{total_corpora}\n\n")
            
            f.write("## Key Findings\n\n")
            
            # Find most effective stopword removal
            if 'Original' in self.results:
                original = self.results['Original']
                reductions = []
                for label, result in self.results.items():
                    if label != 'Original':
                        reduction = (1 - result['total_terms'] / original['total_terms']) * 100
                        quality = len(result['khmer_top_terms'])
                        reductions.append((label, reduction, quality))
                
                reductions.sort(key=lambda x: x[1], reverse=True)
                
                f.write("### Most Effective Stopword Groups\n\n")
                f.write("| Rank | Group | Term Reduction | Quality (Khmer/100) |\n")
                f.write("|------|-------|----------------|---------------------|\n")
                for i, (label, reduction, quality) in enumerate(reductions[:5], 1):
                    f.write(f"| {i} | {label.replace('No_', '')} | {reduction:.1f}% | {quality}/100 |\n")
                f.write("\n")
            
            f.write("## Quality Assessment\n\n")
            for label, result in self.results.items():
                khmer_pct = len(result['khmer_top_terms']) / 100 * 100
                status = "✅ Excellent" if khmer_pct >= 90 else "✓ Good" if khmer_pct >= 80 else "⚠️ Needs review"
                
                f.write(f"### {label}\n")
                f.write(f"- **Status**: {status}\n")
                f.write(f"- **Khmer terms in top 100**: {len(result['khmer_top_terms'])}/100\n")
                f.write(f"- **Total terms**: {result['total_terms']:,}\n")
                f.write(f"- **Vocabulary size**: {result['vocab_size']:,}\n")
                
                if result['non_khmer_top_terms']:
                    f.write(f"- **⚠️ Non-Khmer terms detected**: {[t[0] for t in result['non_khmer_top_terms'][:5]]}\n")
                
                f.write("\n")
            
            f.write("## Recommendations\n\n")
            f.write("1. **For Information Retrieval**: Use corpora with highest Khmer term retention\n")
            f.write("2. **For Efficiency**: Balance term reduction with quality preservation\n")
            f.write("3. **Data Quality**: Review corpora with <80 Khmer terms in top 100\n")
            f.write("4. **Further Analysis**: Examine top_terms_*.txt files for domain relevance\n\n")
            
            f.write("## Files Generated\n\n")
            f.write("- `tfidf_summary.csv` - Overall statistics\n")
            f.write("- `vocabulary_overlap.csv` - Term overlap between corpora\n")
            f.write("- `stopword_impact.csv` - Impact analysis of each stopword group\n")
            f.write("- `top_terms_*.txt` - Top 100 terms for each corpus\n")
            f.write("- `corpus_statistics.png` - Visual comparison charts\n")
            f.write("- `tfidf_distributions.png` - Score distribution histograms\n")
            f.write("- `quality_check.png` - Khmer vs non-Khmer term quality\n")
        
        print(f"✓ Final report saved to: {os.path.basename(report_file)}")

# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    # Configuration
    processed_dir = r'D:\Year 5\S1\Information Retrieval\StopwordProject\khmer_stopword_project\data\processed'
    output_dir = r'D:\Year 5\S1\Information Retrieval\StopwordProject\khmer_stopword_project\data\tfidf_results_clean'
    
    # Files to analyze
    files_to_analyze = [
        ('Original', 'original_segmented_sentences.txt'),
        ('No_All_Stopwords', 'no_all_stopwords.txt'),
        ('No_Auxiliary_Verbs', 'no_Auxiliary_Verbs___Aspect_Markers.txt'),
        ('No_Conjunctions', 'no_Conjunctions.txt'),
        ('No_Determiners', 'no_Determiners_and_Quantifiers.txt'),
        ('No_Function_Nouns', 'no_Function_Nouns.txt'),
        ('No_Numbers', 'no_Numbers_and_Time_Expressions.txt'),
        ('No_Particles', 'no_Particles_and_Discourse_Markers.txt'),
        ('No_Politeness', 'no_Politeness_and_Honorifics.txt'),
        ('No_Prepositions', 'no_Prepositions___Relational_Words.txt'),
        ('No_Pronouns', 'no_Pronouns.txt'),
        ('No_Questions', 'no_Question_and_Negation_Words.txt')
    ]
    
    # Initialize analyzer
    print("="*80)
    print(" COMPREHENSIVE TF-IDF ANALYSIS WITH QUALITY FILTERING")
    print(" Enhanced version with Khmer-only token extraction")
    print("="*80)
    
    analyzer = KhmerTfidfAnalyzer(processed_dir, output_dir)
    
    # Run analysis
    analyzer.analyze_all_files(files_to_analyze)
    analyzer.generate_comparative_analysis()

 COMPREHENSIVE TF-IDF ANALYSIS WITH QUALITY FILTERING
 Enhanced version with Khmer-only token extraction

 COMPREHENSIVE TF-IDF ANALYSIS

────────────────────────────────────────────────────────────────────────────────
Loading: Original
────────────────────────────────────────────────────────────────────────────────
✓ Loaded 58,409 documents
  Sample kept tokens: ['សេចក្តីថ្លែងការណ៍', 'ពាក់ព័ន្ធ', 'នឹង', 'ព័ត៌មាន', 'នេះ', 'បាន', 'ឲ្យដឹង', 'ថា', 'លោក', 'នឹង']
  Original tokens: 1,703,731
  After cleaning: 1,691,824
  Removed: 0.7% (non-Khmer content)
  Documents retained: 58,398/58,409

────────────────────────────────────────────────────────────────────────────────
TOKEN QUALITY DIAGNOSTIC: Original
────────────────────────────────────────────────────────────────────────────────
Token length distribution (first 100 docs):
  1 character:       0 tokens (  0.0%)
  2 characters:    314 tokens ( 11.3%)
  3+ characters:  2,458 tokens ( 88.7%)

  Sample of 3+ character tokens (will be kept):